I worked on some California Census Data, using various features of an individual to predict what class of income they belong in (>50k or <=50k). 

Here is some information about the data:

<table>
<thead>
<tr>
<th>Column Name</th>
<th>Type</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td>age</td>
<td>Continuous</td>
<td>The age of the individual</td>
</tr>
<tr>
<td>workclass</td>
<td>Categorical</td>
<td>The type of employer the  individual has (government,  military, private, etc.).</td>
</tr>
<tr>
<td>fnlwgt</td>
<td>Continuous</td>
<td>The number of people the census  takers believe that observation  represents (sample weight). This  variable will not be used.</td>
</tr>
<tr>
<td>education</td>
<td>Categorical</td>
<td>The highest level of education  achieved for that individual.</td>
</tr>
<tr>
<td>education_num</td>
<td>Continuous</td>
<td>The highest level of education in  numerical form.</td>
</tr>
<tr>
<td>marital_status</td>
<td>Categorical</td>
<td>Marital status of the individual.</td>
</tr>
<tr>
<td>occupation</td>
<td>Categorical</td>
<td>The occupation of the individual.</td>
</tr>
<tr>
<td>relationship</td>
<td>Categorical</td>
<td>Wife, Own-child, Husband,  Not-in-family, Other-relative,  Unmarried.</td>
</tr>
<tr>
<td>race</td>
<td>Categorical</td>
<td>White, Asian-Pac-Islander,  Amer-Indian-Eskimo, Other, Black.</td>
</tr>
<tr>
<td>gender</td>
<td>Categorical</td>
<td>Female, Male.</td>
</tr>
<tr>
<td>capital_gain</td>
<td>Continuous</td>
<td>Capital gains recorded.</td>
</tr>
<tr>
<td>capital_loss</td>
<td>Continuous</td>
<td>Capital Losses recorded.</td>
</tr>
<tr>
<td>hours_per_week</td>
<td>Continuous</td>
<td>Hours worked per week.</td>
</tr>
<tr>
<td>native_country</td>
<td>Categorical</td>
<td>Country of origin of the  individual.</td>
</tr>
<tr>
<td>income</td>
<td>Categorical</td>
<td>"&gt;50K" or "&lt;=50K", meaning  whether the person makes more  than \$50,000 annually.</td>
</tr>
</tbody>
</table>

### THE DATA

In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
census = pd.read_csv("C:\\Users\\dell\\Desktop\\census_data.csv")

In [3]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
census.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 14 columns):
age               32561 non-null int64
workclass         32561 non-null object
education         32561 non-null object
education_num     32561 non-null int64
marital_status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
gender            32561 non-null object
capital_gain      32561 non-null int64
capital_loss      32561 non-null int64
hours_per_week    32561 non-null int64
native_country    32561 non-null object
income_bracket    32561 non-null object
dtypes: int64(5), object(9)
memory usage: 3.5+ MB


## Converting output to Numeric Form

In [5]:
census['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [6]:
def label_fix(label):
    if label==' <=50K':
        return 0
    else:
        return 1

In [7]:
census['income_bracket'] = census['income_bracket'].apply(label_fix)

### Perform a Train Test Split on the Data

In [8]:
x_data = census.drop('income_bracket',axis=1)
y_labels = census['income_bracket']

X_train, X_test, y_train, y_test = train_test_split(x_data,y_labels,test_size=0.3,random_state=101)

### Working on tf.esitmator


In [9]:
census.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

#### Create numeric columns for the categorical values.

In [10]:
gender = tf.feature_column.categorical_column_with_vocabulary_list("gender", ["Female", "Male"])
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)

** Create the continuous feature_columns for the continuous values using numeric_column **

In [11]:
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

** Put all these variables into a single list with the variable name feat_cols **

In [12]:
feat_cols = [gender, occupation, marital_status, relationship, education, workclass, native_country,
            age, education_num, capital_gain, capital_loss, hours_per_week]

### Create Input Function

In [13]:
input_func=tf.estimator.inputs.pandas_input_fn(x = X_train, y = y_train, batch_size = 100, num_epochs = None, shuffle = True)

#### Model using Linear Classifier..Can also be done by DNN Classifier


In [14]:
model = tf.estimator.LinearClassifier(feature_columns = feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_task_id': 0, '_master': '', '_service': None, '_num_worker_replicas': 1, '_num_ps_replicas': 0, '_model_dir': 'C:\\Users\\dell\\AppData\\Local\\Temp\\tmpu0vhx7um', '_task_type': 'worker', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000025E2ACDCF28>, '_log_step_count_steps': 100, '_save_checkpoints_secs': 600, '_session_config': None, '_is_chief': True, '_tf_random_seed': None}


** Train your model on the data, for at least 5000 steps. **

In [15]:
model.train(input_fn = input_func, steps = 5000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\dell\AppData\Local\Temp\tmpu0vhx7um\model.ckpt.
INFO:tensorflow:step = 1, loss = 69.3147
INFO:tensorflow:global_step/sec: 125.853
INFO:tensorflow:step = 101, loss = 669.476 (0.798 sec)
INFO:tensorflow:global_step/sec: 203.108
INFO:tensorflow:step = 201, loss = 108.277 (0.495 sec)
INFO:tensorflow:global_step/sec: 154.212
INFO:tensorflow:step = 301, loss = 534.312 (0.646 sec)
INFO:tensorflow:global_step/sec: 158.87
INFO:tensorflow:step = 401, loss = 144.605 (0.631 sec)
INFO:tensorflow:global_step/sec: 208.838
INFO:tensorflow:step = 501, loss = 38.2653 (0.481 sec)
INFO:tensorflow:global_step/sec: 210.377
INFO:tensorflow:step = 601, loss = 115.867 (0.472 sec)
INFO:tensorflow:global_step/sec: 213.296
INFO:tensorflow:step = 701, loss = 150.222 (0.468 sec)
INFO:tensorflow:global_step/sec: 202.081
INFO:tensorflow:step = 801, loss = 41.4633 (0.495 sec)
INFO:tensorflow:global_step/sec: 180.699
INFO

### Evaluation

In [16]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x = X_test,batch_size = len(X_test), shuffle = False)

In [17]:
predictions = list(model.predict(input_fn = pred_fn))

INFO:tensorflow:Restoring parameters from C:\Users\dell\AppData\Local\Temp\tmpu0vhx7um\model.ckpt-5000


In [18]:
predictions[0]

{'class_ids': array([0], dtype=int64),
 'classes': array([b'0'], dtype=object),
 'logistic': array([ 0.28798336], dtype=float32),
 'logits': array([-0.90519857], dtype=float32),
 'probabilities': array([ 0.71201658,  0.28798333], dtype=float32)}

In [19]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

In [20]:
final_preds[:10]

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

## Classification Report

In [21]:
print(classification_report(y_test, final_preds))

             precision    recall  f1-score   support

          0       0.89      0.90      0.89      7436
          1       0.66      0.64      0.65      2333

avg / total       0.83      0.84      0.83      9769



# task complete!....................   83% Accuracy using Neural Nets